In [357]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import numpy as np
from sklearn import tree
import sklearn.linear_model as lm
from sklearn import preprocessing, model_selection, pipeline
from sklearn.model_selection import validation_curve
from sklearn import metrics
from sklearn.inspection import permutation_importance

from sklearn import svm



from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets


In [362]:
np.set_printoptions(precision=3)
pandas.set_option("display.precision", 3)



classification_scores = ['accuracy','precision_weighted',
                         'recall_weighted', 'roc_auc_ovr_weighted',
                         'f1_weighted']

%matplotlib inline

<font size="6"><div align="center"><span style="color:blue">Leitura da Base</span></div></font>

In [365]:
df_churn = pandas.read_csv('churn.csv')
churn_target_col = 'churn_risk_score'
churn_label_map = df_churn[['churn_risk_score', 'churn_risk_score']].drop_duplicates()

In [367]:
drop_cols = ['churn_risk_score']
#df_churn.drop(drop_cols, axis=1, inplace=True)
df_churn.head()

,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1


In [368]:
print(df_churn.shape)
df_churn.head()

(36992, 24)


,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1


<font size="6"><div align="center"><span style="color:blue">Transformação Variáveis Categóricas</span></div></font>

In [369]:
df_churn['gender']=df_churn['gender'].map({'F':'0',
                             'M':'1'},
                             na_action=None)

df_churn['used_special_discount']=df_churn['used_special_discount'].map({'Yes':'1',
                             'No':'0'},
                             na_action=None)

df_churn['offer_application_preference']=df_churn['offer_application_preference'].map({'Yes':'1',
                             'No':'0'},
                             na_action=None)

df_churn['past_complaint']=df_churn['past_complaint'].map({'Yes':'1',
                             'No':'0'},
                             na_action=None)


categoria_idade = pandas.cut(df_churn.age,bins=[0,17,25,35,40,60,200],
                                labels=['Criança','Adulto inicial',
                                        'Adulto pleno','Adulto final',
                                        'Meia-idade','Idoso'])
df_churn.insert(2,'age_group',categoria_idade)

df_churn.head()

,Unnamed: 0,age,age_group,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,Adulto inicial,0,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,...,300.63,53005.25,17.0,781.75,1,1,0,Not Applicable,Products always in Stock,0
1,1,32,Adulto pleno,0,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,...,306.34,12838.38,10.0,NaN,1,0,1,Solved,Quality Customer Care,0
2,2,44,Meia-idade,0,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,...,516.16,21027.00,22.0,500.69,0,1,1,Solved in Follow-up,Poor Website,1
3,3,37,Adulto final,1,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,...,53.27,25239.56,6.0,567.66,0,1,1,Unsolved,Poor Website,1
4,4,31,Adulto pleno,0,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,...,113.13,24483.66,16.0,663.06,0,1,1,Solved,Poor Website,1


In [370]:

#df_churn.dropna(subset = ['region_category'], inplace=True)
#df_churn.points_in_wallet = df_churn.points_in_wallet.fillna(0)
#df_churn.dtypes

In [371]:
categorical_cols = ['membership_category','age_group','complaint_status','feedback']
encoder_map = {}

for cname in categorical_cols:
    encoder = preprocessing.OneHotEncoder(sparse=False)
    transformed = encoder.fit_transform(df_churn[[cname]])
    ohe_df = pandas.DataFrame(transformed, columns=[cname+'_'+cat for cat in encoder.categories_[0]])
    encoder_map[cname] = encoder
    
    df_churn = pandas.concat([df_churn, ohe_df], axis=1).drop(cname, axis=1)
    

    
df_churn.head()


,Unnamed: 0,age,gender,security_no,region_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,...,complaint_status_Unsolved,feedback_No reason specified,feedback_Poor Customer Service,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website
0,0,18,0,XW0DQ7H,Village,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,32,0,5K0N3X1,City,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,44,0,1F2TCL3,Town,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,37,1,VJGJ33N,City,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,31,0,SVZXCWB,City,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [372]:
df_churn.dropna(subset = ['region_category'], inplace=True)
df_churn.points_in_wallet = df_churn.points_in_wallet.fillna(0)
df_churn.dtypes

categorical_cols = ['region_category']
encoder_map = {}

for cname in categorical_cols:
    encoder = preprocessing.OneHotEncoder(sparse=False)
    transformed = encoder.fit_transform(df_churn[[cname]])
    ohe_df = pandas.DataFrame(transformed, columns=[cname+'_'+cat for cat in encoder.categories_[0]])
    encoder_map[cname] = encoder
    
    df_churn = pandas.concat([df_churn, ohe_df], axis=1).drop(cname, axis=1)
    

    
df_churn.head()


,Unnamed: 0,age,gender,security_no,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,...,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website,region_category_City,region_category_Town,region_category_Village
0,0.0,18.0,0,XW0DQ7H,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,Wi-Fi,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,32.0,0,5K0N3X1,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,Mobile_Data,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,44.0,0,1F2TCL3,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,Wi-Fi,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3.0,37.0,1,VJGJ33N,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,Mobile_Data,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4.0,31.0,0,SVZXCWB,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,Mobile_Data,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


<font size="6"><div align="center"><span style="color:blue">Treinamento dos modelos</span></div></font>

In [375]:
Y = df_churn[churn_target_col]
X = df_churn.drop(churn_target_col,axis=1)

# train/test
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, Y, test_size=0.2)

cvfold = model_selection.StratifiedKFold(n_splits = 10, random_state = 0, shuffle=True)

<font size="6"><div align="center"><span style="color:blue">Regressão Logística</span></div></font>

<font size="4"><div align="Left"><span style="color:blue">Treinamento do modelo
</span></div></font>

In [382]:
model_map = {
    'log_l1': lm.LogisticRegression(penalty='l1', solver='liblinear'),
    'log_l2': lm.LogisticRegression(penalty='l2', solver='liblinear'),
    'log_el': lm.LogisticRegression(penalty='elasticnet',l1_ratio= 0.4, solver='saga'),
}

# solver : 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'

Y = df_churn[churn_target_col]
X = df_churn.drop(churn_target_col, axis=1)

# train/test
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, Y, test_size=0.2)

df_train = xtrain.copy()
df_test = xtest.copy()
df_train['train_set'] = 1
df_test['train_set'] = 0
df_train[churn_target_col] = ytrain
df_test[churn_target_col] = ytest

for modelname, model in model_map.items():
    model.fit(xtrain, ytrain)      
    yhat_train = model.predict(xtrain)
    yhat_test  = model.predict(xtest)
    # Output table
    df_train[modelname] = yhat_train
    df_test[modelname] = yhat_test
    model_map[modelname] = model
    
df_wine = pandas.concat((df_train, df_test), axis=0)

ValueError: could not convert string to float: 'DZ9Y80D'